In [15]:
import numpy as np
import seaborn as sns
import timm
import torch

sns.set_theme()
%load_ext autoreload
%autoreload 2

In [16]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

torch.manual_seed(42)

Need a model that takes a sequence of images as input and chooses two among them for the best exposure fusion result

In [17]:
comparator = max
best_score = np.nan
items = [1, 10, 0.03, 0.2]
for item in items:
    best_score = comparator(item, best_score)

print(best_score)

10


In [18]:
best_score

10

In [19]:
import pandas as pd

In [20]:
import json
from pathlib import Path

p = Path("./teststore.json")
D = dict()
for i in range(10):
    d = tuple((f"alalal{i}", tuple(i + j for j in range(4))))
    entry = dict(entry3=i + 1919, entry4=i * i)
    D[d] = entry
    res = {**dict(entry1=d[0], entry2=d[1]), **entry}
    with p.open("a") as s:
        json.dump(res, s, separators=(",", ":"))
        s.write("\n")

In [21]:
df = pd.read_json(p, lines=True, orient="records", dtype={"entry2": list})
asdict = df.to_dict(orient="records")

In [22]:
d2 = dict()
for row in asdict:
    key = tuple((row["entry1"], tuple(row["entry2"])))
    d2[key] = {k: row[k] for k in ("entry3", "entry4")}

d2

{('alalal0', (0, 1, 2, 3)): {'entry3': 1919, 'entry4': 0},
 ('alalal1', (1, 2, 3, 4)): {'entry3': 1920, 'entry4': 1},
 ('alalal2', (2, 3, 4, 5)): {'entry3': 1921, 'entry4': 4},
 ('alalal3', (3, 4, 5, 6)): {'entry3': 1922, 'entry4': 9},
 ('alalal4', (4, 5, 6, 7)): {'entry3': 1923, 'entry4': 16},
 ('alalal5', (5, 6, 7, 8)): {'entry3': 1924, 'entry4': 25},
 ('alalal6', (6, 7, 8, 9)): {'entry3': 1925, 'entry4': 36},
 ('alalal7', (7, 8, 9, 10)): {'entry3': 1926, 'entry4': 49},
 ('alalal8', (8, 9, 10, 11)): {'entry3': 1927, 'entry4': 64},
 ('alalal9', (9, 10, 11, 12)): {'entry3': 1928, 'entry4': 81}}

In [23]:
D

{('alalal0', (0, 1, 2, 3)): {'entry3': 1919, 'entry4': 0},
 ('alalal1', (1, 2, 3, 4)): {'entry3': 1920, 'entry4': 1},
 ('alalal2', (2, 3, 4, 5)): {'entry3': 1921, 'entry4': 4},
 ('alalal3', (3, 4, 5, 6)): {'entry3': 1922, 'entry4': 9},
 ('alalal4', (4, 5, 6, 7)): {'entry3': 1923, 'entry4': 16},
 ('alalal5', (5, 6, 7, 8)): {'entry3': 1924, 'entry4': 25},
 ('alalal6', (6, 7, 8, 9)): {'entry3': 1925, 'entry4': 36},
 ('alalal7', (7, 8, 9, 10)): {'entry3': 1926, 'entry4': 49},
 ('alalal8', (8, 9, 10, 11)): {'entry3': 1927, 'entry4': 64},
 ('alalal9', (9, 10, 11, 12)): {'entry3': 1928, 'entry4': 81}}

In [30]:
from dhdrnet.dataset import CachingDataset

In [31]:
ds = CachingDataset(data_dir=Path("../../data"))

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/shane/.cache/pypoetry/virtualenvs/dhdrnet-CxDQYEDG-py3.8/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


In [32]:
ds.data_generator.best_evs

{}

In [44]:
ims, d = ds[1000]
ims[0].shape, d

((379, 506, 3), {'score': 31.410558923299902, 'evs': (-2, 4)})

In [38]:
score

'score'

In [40]:
d

{'score': 24.749260146463783, 'evs': (0, 4)}